# Safe Search Detection

This demo use **Google Cloud Platform Vision API**.

In [1]:
import time 
import urllib
import requests
import cv2
import operator
import numpy as np
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline 

_url = 'https://vision.googleapis.com/v1/images:annotate'
_key = '__KEY__'

In [2]:
def processRequest( json, data, headers, params ):

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result

In [4]:
from IPython.display import Image as ipImage, display

urlImage = 'http://cdn.ofuxico.com.br/img/upload/noticias/2017/05/11/nua-kourtney-kardashian-faz-ensaio-sensual-em-piscina_293878_36.jpg'

img = ipImage(url=urlImage, width=200)
display(img)

In [5]:
headers = dict()
headers['Content-Type'] = 'application/json' 

json = {
  "requests": [
    {
      "image": {
        "source": {
          "imageUri": urlImage
        }
      },
      "features": [
        {
          "type": "SAFE_SEARCH_DETECTION"
        }
      ]
    }
  ]
}

data = None
params = urllib.parse.urlencode({
    'key': _key
})

result = processRequest( json, data, headers, params )

In [6]:
import json

print(json.dumps(result, indent=2, sort_keys=True))

{
  "responses": [
    {
      "safeSearchAnnotation": {
        "adult": "POSSIBLE",
        "medical": "VERY_UNLIKELY",
        "spoof": "VERY_UNLIKELY",
        "violence": "VERY_UNLIKELY"
      }
    }
  ]
}
